In [0]:
from psutil import *

In [0]:
cpu_count()

2

In [0]:
cpu_stats()

scpustats(ctx_switches=270969, interrupts=163141, soft_interrupts=117502, syscalls=0)

In [0]:
virtual_memory()

svmem(total=13655232512, available=12778442752, percent=6.4, used=586158080, free=10913701888, active=732557312, inactive=1725415424, buffers=76042240, cached=2079330304, shared=921600, slab=162533376)

In [0]:
# !cat /proc/cpuinfo

In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!dir

drive  sample_data


In [6]:
%cd drive/My\ Drive/Colab\ Notebooks/Punctuation_Restoration/BertPunc-master_Colab_without_padding_and_mirror_change_first_and_last_tokens

/content/drive/My Drive/Colab Notebooks/Punctuation_Restoration/BertPunc-master_Colab_without_padding_and_mirror_change_first_and_last_tokens


In [7]:
!dir

7k_without_punctuation.txt	 LICENSE       train2012_MAX_fix.txt
BertPunc_TRAINED_ALL_LAYER.txt	 model.py      train2012_MAX.txt
BertPunc_TRAINED_TOP_LAYER.txt	 models        training.ipynb
data				 __pycache__   train.py
data.py				 README.md     train.txt
evaluate.ipynb			 results
input_without_second_column.txt  row_text.txt


In [8]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 40.4MB/s 


TRAINING ALL LAYERS (1 epoch)

    epochs_all = 1  # by default 4
    iterations_all = 1 # by default 3 (one iteration = 2 hours 30 min)
    batch_size_all = 200 # by default 256
    learning_rate_all = 1e-5
    best_val_loss=1e9

In [0]:
import time
start_time = time.time()
!python train.py
elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

90
LOADING DATA...
PREPROCESSING DATA...
my method
my method
INITIALIZING MODEL...
TRAINING ALL LAYERS...
  7% 1854/25461 [28:16<6:02:13,  1.09it/s]

# Checking different ways to write a insert_target function

In [0]:
import numpy as np

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
punctuation_enc = {
    'O': 0,
    'COMMA': 1,
    'PERIOD': 2,
    'QUESTION': 3
}

In [0]:
def load_file(filename):
    with open(filename, 'r', encoding="utf8", errors="ignore") as f:  # , encoding='utf-8'
        data = f.readlines()
    return data

In [0]:
data = load_file('train.txt')

In [0]:
def encode_data(data, tokenizer, punctuation_enc):

    X = []
    Y = []
    for line in data:
        word, punc = line.split('\t')
        punc = punc.strip()
        tokens = tokenizer.tokenize(word)
        x = tokenizer.convert_tokens_to_ids(tokens)
        y = [punctuation_enc[punc]]
        if len(x) > 0:
            if len(x) > 1:
                y = (len(x)-1)*[0]+y
            X += x
            Y += y
    return X, Y

In [0]:
X, y = encode_data(data, tokenizer, punctuation_enc)

In [0]:
def insert_target(x, segment_size):

    X = []
    x_pad = x[-((segment_size-1)//2-1):]+x+x[:segment_size//2]

    for i in range(len(x_pad)-segment_size+2):
        segment = x_pad[i:i+segment_size-1]
        # segment.insert((segment_size-1)//2, 0)
        X.append(segment)

    # x_segment = []
    # for i in range(len(x)):
    #     if i > segment_size and i < len(x) - segment_size:
    #         x_segment.append(x[segment_size:i]+x[i]+x[i:segment_size])
    

    return np.array(X)

In [0]:
insert_target(X, 32)[0]

array([ 2360,  2242,  2055,  1996,  5505,  1038,  3501,  2057,  2018,
        1037,  2502,  4119,  2007,  5505,  1045,  2031,  2019,  2324,
        1011,  2117, 12528,  1997,  1996,  2200,  2034,  3328,  1997,
        1996,  8773,  2023,  2003])

In [0]:
def insert_target(x, segment_size):

    X = []
    x_pad = x[-((segment_size-1)//2-1):]+x+x[:segment_size//2]

    for i in range(len(x_pad)-segment_size+2):
        segment = x_pad[i:i+segment_size-1]
        segment.insert((segment_size-1)//2, 0)
        X.append(segment)
 
    return np.array(X)

In [0]:
insert_target(X, 32)[0]

array([ 2360,  2242,  2055,  1996,  5505,  1038,  3501,  2057,  2018,
        1037,  2502,  4119,  2007,  5505,  1045,     0,  2031,  2019,
        2324,  1011,  2117, 12528,  1997,  1996,  2200,  2034,  3328,
        1997,  1996,  8773,  2023,  2003])

In [0]:
def insert_target(x, segment_size):

    X = []
    x_pad = x[-((segment_size-1)//2-1):]+x+x[:segment_size//2]

    for i in range(len(x_pad)-segment_size+2):
        segment = x_pad[i:i+segment_size-1]
        segment.insert((segment_size-1)//2, 0)
        X.append(segment)
 
    return np.array(X)

In [0]:
insert_target(X, 32)[0]

array([ 2242,  2055,  1996,  5505,  1038,  3501,  2057,  2018,  1037,
        2502,  4119,  2007,  5505,  1045,  2031,     0,  2019,  2324,
        1011,  2117, 12528,  1997,  1996,  2200,  2034,  3328,  1997,
        1996,  8773,  2023,  2003,  1999])

In [0]:
def insert_target(x, segment_size):

    X = []
    x_pad = x[-(((segment_size+1)-1)//2-1):]+x+x[:(segment_size+1)//2]

    for i in range(len(x_pad)-(segment_size+1)+2):
        segment = x_pad[i:i+segment_size+1-1]
        # segment.insert((segment_size-1)//2, 0)
        X.append(segment)
 
    return np.array(X)

In [0]:
insert_target(X, 32)[0]

array([ 2000,  2360,  2242,  2055,  1996,  5505,  1038,  3501,  2057,
        2018,  1037,  2502,  4119,  2007,  5505,  1045,  2031,  2019,
        2324,  1011,  2117, 12528,  1997,  1996,  2200,  2034,  3328,
        1997,  1996,  8773,  2023,  2003])

In [0]:
def insert_target(x, segment_size):

 
    x_segment = []
    for i in range(len(x)):
        if i >= segment_size//2 and i <= len(x) - segment_size//2:
          #x_segment.append([x[i]]+x[i:segment_size])
          x_segment.append(x[i-segment_size//2:i+segment_size//2])
        if i < segment_size//2:
          x_segment.append(x[:segment_size])
        if i > len(x) - segment_size//2:
          x_segment.append(x[len(x)-segment_size:])

  
    return np.array(x_segment)

In [0]:
print(len(X))
print(len(insert_target(X, 32)))
print(len(insert_target(X, 32)[0]))
print(len(insert_target(X, 32)[1]))
print(len(insert_target(X, 32)[100]))
print(len(insert_target(X, 32)[534]))
print(len(insert_target(X, 32)[535]))

538
538
32
32
32
32
32


In [0]:
print(X[:100])

[1045, 2031, 2019, 2324, 1011, 2117, 12528, 1997, 1996, 2200, 2034, 3328, 1997, 1996, 8773, 2023, 2003, 1999, 1996, 2120, 3004, 2996, 1996, 2173, 2073, 2027, 5660, 2047, 4784, 2009, 2018, 2011, 2053, 2965, 2288, 1996, 2665, 2422, 2664, 1996, 17334, 11291, 7367, 2094, 2290, 7184, 8826, 1037, 3376, 5537, 2073, 1996, 3336, 3586, 2029, 2001, 2081, 2041, 1997, 12668, 1998, 9017, 1997, 27964, 3473, 2039, 2046, 1996, 2502, 3586, 1998, 4172, 14330, 1996, 2472, 1997, 1996, 2120, 3004, 2387, 2008, 3327, 2617, 2002, 2001, 3061, 2279, 2000, 2033, 2002, 3053, 4954, 2370, 1998, 2061, 1996, 2265, 2001, 2445, 1996]


In [0]:
print(insert_target(X, 6)[0])
print(insert_target(X, 6)[1])
print(insert_target(X, 6)[2])
print(insert_target(X, 6)[3])
print(insert_target(X, 6)[4])
print(insert_target(X, 6)[5])
print(insert_target(X, 6)[6])
print(insert_target(X, 6)[7])
print(insert_target(X, 6)[8])

[1045 2031 2019 2324 1011 2117]
[1045 2031 2019 2324 1011 2117]
[1045 2031 2019 2324 1011 2117]
[1045 2031 2019 2324 1011 2117]
[ 2031  2019  2324  1011  2117 12528]
[ 2019  2324  1011  2117 12528  1997]
[ 2324  1011  2117 12528  1997  1996]
[ 1011  2117 12528  1997  1996  2200]
[ 2117 12528  1997  1996  2200  2034]


In [0]:
print(X[-20:])

[5725, 3670, 17712, 2017, 2215, 2000, 2360, 2242, 2055, 1996, 5505, 1038, 3501, 2057, 2018, 1037, 2502, 4119, 2007, 5505]


In [0]:
print(insert_target(X, 6)[-9])
print(insert_target(X, 6)[-8])
print(insert_target(X, 6)[-7])
print(insert_target(X, 6)[-6])
print(insert_target(X, 6)[-5])
print(insert_target(X, 6)[-4])
print(insert_target(X, 6)[-3])
print(insert_target(X, 6)[-2])
print(insert_target(X, 6)[-1])

[2055 1996 5505 1038 3501 2057]
[1996 5505 1038 3501 2057 2018]
[5505 1038 3501 2057 2018 1037]
[1038 3501 2057 2018 1037 2502]
[3501 2057 2018 1037 2502 4119]
[2057 2018 1037 2502 4119 2007]
[2018 1037 2502 4119 2007 5505]
[2018 1037 2502 4119 2007 5505]
[2018 1037 2502 4119 2007 5505]
